In [2]:
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Got Libraries')

Solving environment: done

# All requested packages already installed.

Got Libraries


In [94]:
address = 'Chennai'
geolocator = Nominatim(user_agent="smy-application")
location = geolocator.geocode(address)
latitude_x = location.latitude
longitude_y = location.longitude
print('The Geograpical Co-ordinate of Chennai are {}, {}.'.format(latitude_x, longitude_y))

The Geograpical Co-ordinate of Chennai are 13.0836939, 80.270186.


In [95]:
neighborhood_latitude=13.0836939
neighborhood_longitude=80.270186

In [111]:
LIMIT = 1050 # limit of number of venues returned by Foursquare API
radius = 1500#1650
CLIENT_ID='ZWLCLXKRKKBH1AXN2EMN0TLRRVCPDINO1CYW4AVA5VUFANK3'
CLIENT_SECRET='2OKBDY3EH23YLKNIS1UEDB4BHBGBLB2SP4EPOABWQ0LCJF5X'
VERSION='20180605' # Foursquare API version

#https://api.foursquare.com/v2/venues/search
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#url = 'https://api.foursquare.com/v2/search/recommendations?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ZWLCLXKRKKBH1AXN2EMN0TLRRVCPDINO1CYW4AVA5VUFANK3&client_secret=2OKBDY3EH23YLKNIS1UEDB4BHBGBLB2SP4EPOABWQ0LCJF5X&v=20180605&ll=13.0836939,80.270186&radius=1500&limit=1050'

In [113]:
results = requests.get(url).json()
results

#items = results['response']['groups'][0]['items']
#items

{'meta': {'code': 200, 'requestId': '60375ed2ff5f9a25f4d20062'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Chennai',
  'headerFullLocation': 'Chennai',
  'headerLocationGranularity': 'city',
  'totalResults': 30,
  'suggestedBounds': {'ne': {'lat': 13.097193913500014,
    'lng': 80.28401993028831},
   'sw': {'lat': 13.070193886499986, 'lng': 80.25635206971168}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4eeb7737aa1f66e35b2cff5d',
       'name': 'National Durbar Hotel',
       'location': {'address': '#3, Raja Muthiah Rd',
        'crossStreet': 'near Chennai Central. Eggmore',
        'lat': 13.081300779830098,
        'lng': 80.27060089302557,
        'labeledLatLngs':

In [129]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [131]:
# to filter Stadium, BookStore and Train Station
options = ['Stadium', 'Bookstore', 'Soccer Stadium','Train Station'] 
venues = results['response']['groups'][0]['items']

#flatten JSON, Normalize JSON to Dataframe
dataframe = json_normalize(venues) 

# filter columns,consider only required columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
nearby = dataframe.loc[:, filtered_columns]

# filter the category for each row
nearby['venue.categories'] = nearby.apply(get_category_type, axis=1)

nearby = nearby[nearby['venue.categories'].isin(options)]

# clean columns
nearby.columns = [col.split('.')[-1] for col in nearby.columns]

#replce NaN values with Not found in address
nearby['address'] = nearby['address'].fillna("Not found")
nearby['crossStreet'] = nearby['address'].fillna("Not found")
nearby

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
8,Higginbothams,Bookstore,"Station Rd, Park Town","Station Rd, Park Town",13.083190,80.275168,"[{'label': 'display', 'lat': 13.08318961077487...",543,NaN,IN,Chennai,Tamil Nadu,India,"[Station Rd, Park Town (Central Station), Chen...",4ce54c53ca41236af87e7b4b
12,Jawaharlal Nehru Stadium.,Soccer Stadium,"Raja Muthiah Rd, Periyamet","Raja Muthiah Rd, Periyamet",13.085253,80.270862,"[{'label': 'display', 'lat': 13.08525342433347...",188,NaN,IN,Chennai,Tamil Nadu,India,"[Raja Muthiah Rd, Periyamet, Chennai, Tamil Na...",4cdc001cdf986ea8465ccd16
14,Chennai Central Station,Train Station,Not found,Not found,13.082762,80.274901,"[{'label': 'display', 'lat': 13.08276218316883...",521,NaN,IN,Chennai,Tamil Nadu,India,"[Chennai, Tamil Nadu, India]",5624c11e498eec470ec9d53e
16,Jawaharlal Nehru Indoor Stadium,Stadium,"Periamet, Chennai","Periamet, Chennai",13.085060,80.269054,"[{'label': 'display', 'lat': 13.08506032243801...",195,600003,IN,Chennai,Tamil Nadu,India,"[Periamet, Chennai (Sydenhams Rd), Chennai 600...",4c1c4891c09ed13aeea4828e
17,Moore Market,Bookstore,near chennai centeal station,near chennai centeal station,13.082200,80.272893,"[{'label': 'display', 'lat': 13.08220020605885...",337,NaN,IN,Chennai,Tamil Nadu,India,"[near chennai centeal station ((sub urban)), C...",4d344669c75a6ea86c1b1eae
22,Park Town,Train Station,Not found,Not found,13.079439,80.275930,"[{'label': 'display', 'lat': 13.07943895164443...",782,NaN,IN,NaN,NaN,India,[India],5263b86d11d201a7964716de


In [148]:
latitude = neighborhood_latitude 
longitude = neighborhood_longitude
map_ch = folium.Map(location=[latitude, longitude], zoom_start=15)

# add markers to map
for lat, lng, label in zip(nearby['lat'], nearby['lng'], nearby['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=10,
        radius=30,
        popup=label,
        color='red',
        fill_color='yellow',
        fill_opacity=1.0,
    ).add_to(map_ch)  
    
map_ch
